这是一个使用 CodeFuse-Query 分析 Go 项目的教程。在教程中，你将体验到使用命令行工具对代码仓库进行数据化，然后使用 Godel 语言来分析这个仓库。

检查cli是否就绪

In [1]:
!which sparrow

/sparrow-cli/sparrow



STEP 0: 克隆要分析的仓库。我们以 [gorm](https://github.com/go-gorm/gorm.git) 项目为例。

In [2]:
!git clone https://github.com/go-gorm/gorm.git -q

STEP 1: 代码数据化。使用 `sparrow database create` 命令创建一个db文件，指定待分析的仓库地址（当前目录下的gorm子目录），分析的语言（go），以及db文件的存储路径（放置在当前目录下的/db/gorm）。执行该命令之后，就会生成一份db文件，该文件存储着代码仓库的结构化数据，之后的分析就是针对这份数据进行。

In [7]:
!sparrow database create --source-root gorm --data-language-type go --output ./db/gorm --overwrite > /dev/null

STEP 2: 使用Godel分析语言分析db文件。在本教程中，可以点击代码左侧的执行按钮，或使用快捷键：`Shift+Enter`，直接运行分析脚本。这里使用 `%db /path/to/db` 魔法命令来设置COREF db路径，内核会读取这个值来进行query查询。

<b>示例</b> 查询 [gorm](https://github.com/go-gorm/gorm.git) 的文件代码复杂度信息。

第一行通过内核魔法命令指定分析的db路径，后面写查询文件代码复杂度Godel脚本。

In [4]:
%db ./db/gorm
// script
use coref::go::*

fn default_db() -> GoDB {
    return GoDB::load("coref_go_src.db")
}

/**
 * @param name: 文件名
 * @param func: 函数名
 * @param cmplx: 函数圈复杂度
 * @param sl,el,sc,ec: 函数位置信息，依次为函数起始行，结束行
 */
fn out(name: string, func: string, cmplx: int, sl: int, el: int) -> bool {
    for(f in GoFile(default_db()), function in Function(default_db())) {
        if ((!f.isAutoGenereatedFile()) &&
            f.key_eq(function.getBelongsFile()) &&
            name = f.getName() &&
            func = function.getName() &&
            cmplx = function.getCyclomaticComplexity() &&
            sl = function.getLocation().getStartLineNumber() &&
            el = function.getLocation().getEndLineNumber()) {
            return true
        }
    }
}

fn main() {
    output(out())
}

/workspaces/CodeFuse-Query/tutorial/notebook/db/gorm


Sparrow database is set to: /workspaces/CodeFuse-Query/tutorial/notebook/db/gorm


2023-12-06 07:42:29,249 INFO: sparrow 2.0.0
 will start
2023-12-06 07:42:29,249 INFO: database /workspaces/CodeFuse-Query/tutorial/notebook/db/gorm/coref_go_src.db size: 42.32 MB
2023-12-06 07:42:29,250 INFO: execute : /sparrow-cli/godel-script/usr/bin/godel /tmp/godel-jupyter-t7lsy12n/query.gdl -p /sparrow-cli/lib-1.0 -o /tmp/tmphp2bm3pi.gdl
2023-12-06 07:42:29,314 INFO: godel-script compile time: 0.06s
2023-12-06 07:42:29,315 INFO: execute : /sparrow-cli/godel-1.0/usr/bin/godel /tmp/tmphp2bm3pi.gdl --run-souffle-directly --package-path /sparrow-cli/lib-1.0 --souffle-fact-dir /workspaces/CodeFuse-Query/tutorial/notebook/db/gorm --souffle-output-format json --souffle-output-path /tmp/godel-jupyter-t7lsy12n/query.json
2023-12-06 07:42:39,655 INFO: Task /tmp/godel-jupyter-t7lsy12n/query.gdl is success, result is NOT-EMPTY, execution time is  10.41s.
2023-12-06 07:42:39,656 INFO: run success

Total results: 942


,name,func,cmplx,sl,el
0,association.go,Delete,20,197,361
1,association.go,Unscoped,1,44,51
2,association.go,Find,2,53,58
3,association.go,Association,4,21,42
4,association.go,Count,2,367,372
...,...,...,...,...,...
937,tests/preload_test.go,TestPreloadEmptyData,5,215,235
938,tests/preload_test.go,TestPreloadGoroutine,2,237,253
939,tests/preload_test.go,TestPreloadWithDiffModel,2,255,271
940,tests/preload_test.go,TestNestedPreloadWithUnscoped,4,273,308


保存上一次运行的 query 结果保存到一个JSON文件

In [5]:
%%save_to ./query.json

Query result saved to /workspaces/CodeFuse-Query/tutorial/notebook/query.json


STEP 3: 好了，你可以针对分析生成的结果，进行进一步的代码分析了，比如你可以结合pandas库，使用刚刚生成的 query.json 实现最大函数复杂度Top 10的排序查询：

In [6]:
%%python
import pandas as pd
data = pd.read_json('./query.json')
data.sort_values('cmplx', ascending=False, inplace=True)
top_10 = data.head(10)
print(top_10)

                      name                       func  cmplx   sl   el
428        schema/field.go                 ParseField    110   99  436
18     callbacks/update.go       ConvertToAssignments     69  131  304
309       schema/schema.go  ParseWithSpecialTableName     68  121  370
130    callbacks/create.go      ConvertToCreateValues     64  217  385
234                scan.go                       Scan     62  125  342
30            statement.go             BuildCondition     61  284  465
429        schema/field.go       setupValuerAndSetter     46  439  969
729  tests/migrate_test.go         TestMigrateColumns     44  517  645
11      callbacks/query.go              BuildQuerySQL     41   33  266
281   migrator/migrator.go              MigrateColumn     41  441  542


Enjoy！